In [1]:
import re
import time
import random
random.seed(9898978)
from copy import deepcopy

from pybtex.database import parse_string
import rispy
from rispy.utils import convert_reference_types
from tqdm import tqdm

import openai
openai.api_key = ''

/Users/mikekestemont/miniconda3/envs/py3.11/lib/python3.11/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.2.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
ptype = 'BOOK'
with open(f'../data/ris-dump/2010s/{ptype}.ris', 'r') as bibliography_file:
    entries = convert_reference_types(rispy.load(bibliography_file))
print(len(entries))
entries[:3]

3891


[{'type_of_reference': 'Whole book',
  'year': '2010',
  'title': 'Schlichting, Liesbeth. Mijn eerste Van Dale themaboekjes. Utrecht: Van Dale, 2010. 288 p.',
  'abstract': 'ISBN: 978-90-6648-640-9 t. e. m. 978-90-6648-645-4. Niet in autopsie.',
  'date': '22-01-2024',
  'authors': ['Schlichting, Liesbeth'],
  'keywords': ['Van Dale Lexicografie'],
  'name_of_database': 'Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL)'},
 {'type_of_reference': 'Whole book',
  'year': '2010',
  'title': 'Bogaert, Mark van. Met woorden verleiden: schrijftips voor uw presentaties, mailings & andere wervende teksten. Leuven: Lannoo Campus, 2010. 239 p.',
  'abstract': 'ISBN: 978-90-209-8952-6. P. 9-8 Eerst dit: haat-liefdeverhouding; p. 233-239 Register.',
  'date': '22-01-2024',
  'authors': ['Bogaert, Mark van'],
  'keywords': ['schrijfadvies',
   'spreken in het openbaar',
   'elektronische post',
   'advertenties'],
  'name_of_database': 'Bibliografie van de Nederlandse Taal- en Li

In [3]:
random.shuffle(entries)
sample = entries[:50]
sample

[{'type_of_reference': 'Whole book',
  'year': '2014',
  'title': 'Sterke verhalen: vijf eeuwen vertelcultuur; [concept en samenstelling] Jeroen Salman; met bijdragen van Roeland Harms en Talitha Verheij; [tekstredactie Yvonne Bleyerveld]. Nijmegen: Vantilt, 2014. 160 p.',
  'abstract': 'ISBN 978-94-6004-168-6. P. 7-8 Voorwoord, door Garrelt Verhoeven en Jeroen Salman; p. 11-21 Ter introductie: het voortleven van verhalen in woord en beeld, door Jeroen Salman; p. 155-157 Literatuur; p. 158 Verantwoording van het beeld- en bronnenmateriaal. Publicatie ter gelegenheid van de tentoonstelling "Sterke verhalen: vijf eeuwen vertelcultuur" bij de Bijzondere Collecties van de Universiteit van Amsterdam, 24 juni t/m 5 oktober 2014. In het voorjaar van 2014 ontving de Universiteit van Amsterdam een belangrijke schenking van een collectie kinderprenten van verzamelaar Nico Boerma. De waardevolle collectie, met tal van uiterst zeldzame prenten, vormt een belangrijke aanvulling op de verzameling po

In [4]:
desc_strs = [' '.join(entry['title'].split()) for entry in sample]
desc_strs[:5]

['Sterke verhalen: vijf eeuwen vertelcultuur; [concept en samenstelling] Jeroen Salman; met bijdragen van Roeland Harms en Talitha Verheij; [tekstredactie Yvonne Bleyerveld]. Nijmegen: Vantilt, 2014. 160 p.',
 'Desnerck, Roland. Grenzeloos Oostends: Oostends voor gevorderden en voor liefhebbers van een taalkundige trip. Damme: Zorro uitgeverij, 2014. 368 p.',
 'T\'Sjoen, Yves. "Zoals een grens op de kaart": Nederlandse literatuur in vergelijkend perspectief: gevalstudies. Gent: Academia Press, 2014. 258 p.',
 'Hooft, P. C. De gedichten; verzorgd en uitgegeven door Johan Koppenol en Ton van Strien; muzikale redactie en toelichting Natascha Veldhorst. Amsterdam: Athenaeum-Polak & Van Gennep, 2012. 869 p.',
 'Donche, Pieter A. V. P. Driehonderd jaar veilingcatalogi (1717-2017) in de bibliotheek "Stichting de Bethune", Marke. Marke: Pieter Donche, 2017. 269 p.']

In [5]:
batch_size = 5
str_batches = [desc_strs[i:i + batch_size] for i in range(0, len(desc_strs), batch_size)]
ris_batches = [entries[i:i + batch_size] for i in range(0, len(desc_strs), batch_size)]
print('#', len(str_batches))
print('#', len(ris_batches))

# 10
# 10


In [6]:
bibtexs = []

promts = {
            'JOUR': 'Can you parse the following, Dutch-language bibliographic \
                            descriptions (each on a separate line) to valid bibtex entries? \
                            The type of publication should be "@article" for all of them.\
                            Just return the bibtex entries please, and nothing else (no explanation). \
                            If you come across a month, please add the month between round brackets to \
                            the number field: e.g. number = {9 (okt)}. Use the keyword `and` to separate \
                            authors, instead of commas. \
                            Make sure that you return the same number of outputs as I provide inputs; \
                            that is very important. Also, make sure that the bibtex-entries which you \
                            return have a unique key.',
            'BOOK': 'Can you parse the following, Dutch-language bibliographic \
                            descriptions (each on a separate line) to valid bibtex entries? \
                            The type of publication should be "@book" for all of them.\
                            Just return the bibtex entries please, and nothing else (no explanation). \
                            Use the keyword `and` to separate authors and editors, instead of commas. \
                            Do not use the bibtex "address" field for the place of publication, e.g. "address = {Amsterdam/Antwerpen}". \
                            Make sure that you return the same number of outputs as I provide inputs; \
                            that is very important. Also, make sure that the bibtex-entries which you \
                            return have a unique key.',
          
        }

def parse_batch(batch, main_prompt):
    messages = [{'role': 'user',
                'content': main_prompt + ' -> ' + '\n'.join(batch)}]
    responses = openai.ChatCompletion.create(model='gpt-3.5-turbo', messages=messages)
    return responses['choices'][0]['message']['content']

In [7]:
updated_ris = []
update_fields = {'JOUR': ['title', 'volume', 'number', 'year', 'pages'],
                 'BOOK': ['title', 'volume', 'publisher', 'address', 'pages', 'series', 'editora', 'translator']}

def update_ris(ris, bibt, ptype):
    ris['notes_abstract'] = ris['title']
    for k in update_fields[ptype]:
        if k == 'pages' and 'pages' in bibt.fields:
            pages = bibt.fields[k].split('-')
            if len(pages) == 2:
                ris['start_page'] = pages[0]
                ris['end_page'] = pages[1]
        else:
            try:
                ris[k] = bibt.fields[k]
            except KeyError:
                pass
    return ris

for ris_batch, str_batch in tqdm(list(zip(ris_batches, str_batches))):
    time.sleep(1)
    bibtexs = parse_batch(str_batch, main_prompt=promts[ptype])
    bibtexs = parse_string(bibtexs, bib_format="bibtex")

    for ris, bibt in zip(ris_batch, bibtexs.entries):
        print('..........................................')
        bibt = bibtexs.entries[bibt]
        print(bibt)
        updated_ris.append(update_ris(ris, bibt, ptype=ptype))


  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:08<01:15,  8.41s/it]

..........................................
Entry('book',
  fields=[
    ('title', 'Sterke verhalen: vijf eeuwen vertelcultuur'), 
    ('contributor', 'Roeland Harms and Talitha Verheij and Yvonne Bleyerveld'), 
    ('publisher', 'Vantilt'), 
    ('year', '2014'), 
    ('pages', '160')],
  persons=OrderedCaseInsensitiveDict([('editor', [Person('Salman, Jeroen')])]))
..........................................
Entry('book',
  fields=[
    ('title', 'Grenzeloos Oostends: Oostends voor gevorderden en voor liefhebbers van een taalkundige trip'), 
    ('publisher', 'Zorro uitgeverij'), 
    ('year', '2014'), 
    ('pages', '368')],
  persons=OrderedCaseInsensitiveDict([('author', [Person('Desnerck, Roland')])]))
..........................................
Entry('book',
  fields=[
    ('title', '"Zoals een grens op de kaart": Nederlandse literatuur in vergelijkend perspectief: gevalstudies'), 
    ('publisher', 'Academia Press'), 
    ('year', '2014'), 
    ('pages', '258')],
  persons=OrderedC

 20%|██        | 2/10 [00:15<00:59,  7.41s/it]

..........................................
Entry('book',
  fields=[
    ('title', 'Limburgse monografieën; jrg. 23, nr. 4 (okt-dec 2012)'), 
    ('year', '2012'), 
    ('publisher', 'K.V.L.S.'), 
    ('pages', '50')],
  persons=OrderedCaseInsensitiveDict([('author', [Person('Sijpe, Luk van de'), Person('Gerits, Jan')])]))
..........................................
Entry('book',
  fields=[
    ('title', 'Wat maak je me nu?!: inspiratieboek over de menukaart als communicatiemiddel'), 
    ('year', '2019'), 
    ('publisher', 'Het Boekenschap'), 
    ('pages', '159')],
  persons=OrderedCaseInsensitiveDict([('author', [Person('Roemaat, Bianca')])]))
..........................................
Entry('book',
  fields=[
    ('title', 'Niederdeutsches Quartett: Groth, Reuter, Brinckman, Fehrs in Flandern und den Niederlanden'), 
    ('year', '2019'), 
    ('publisher', 'Autumnus Verlag'), 
    ('pages', '144')],
  persons=OrderedCaseInsensitiveDict([('author', [Person('Simons, Ludo')])]))
.....

...

..........................................
Entry('book',
  fields=[
    ('title', 'Vielfalt, Variation und Stellung der deutschen Sprache'), 
    ('year', '2013'), 
    ('publisher', 'De Gruyter'), 
    ('pages', '578')],
  persons=OrderedCaseInsensitiveDict([('author', [Person('Schneider-Wiejowski, Karina'), Person('Kellermeier-Rehbein, Birte'), Person('Haselhuber, Jakob')])]))
..........................................
Entry('book',
  fields=[
    ('title', 'Lunchpauzegedichten'), 
    ('year', '2014'), 
    ('publisher', 'De Bezige Bij/Lebowski'), 
    ('address', 'Amsterdam/Antwerpen'), 
    ('pages', '62')],
  persons=OrderedCaseInsensitiveDict([('author', [Person('Arends, Jan')])]))
..........................................
Entry('book',
  fields=[
    ('title', 'Rethinking cultural transfer and transmission: reflections and new perspectives'), 
    ('assistant_editor', 'Karina Smits'), 
    ('year', '2012'), 
    ('publisher', 'Barkhuis'), 
    ('address', 'Groningen'), 
    ('

 80%|████████  | 8/10 [01:11<00:17,  8.79s/it]

..........................................
Entry('book',
  fields=[
    ('title', 'Naturalistische Moderne - Arno Holz und Lodewijk van Deyssel'), 
    ('year', '2015'), 
    ('publisher', 'Nodus Publikationen')],
  persons=OrderedCaseInsensitiveDict([('author', [Person('Pols, Gijsbert')])]))
..........................................
Entry('book',
  fields=[
    ('title', 'Woekering en weigering: metamorfosen en identiteit in het werk van Jacq Vogelaar'), 
    ('year', '2018'), 
    ('publisher', 'Academia Press'), 
    ('series', 'SEL-reeks; 11')],
  persons=OrderedCaseInsensitiveDict([('editor', [Person('Demeyer, Hans'), Person('Vitse, Sven')])]))
..........................................
Entry('book',
  fields=[
    ('title', 'Vale!: afscheidsgedichten van Willem Bilderdijk'), 
    ('year', '2016'), 
    ('publisher', 'EON Pers')],
  persons=OrderedCaseInsensitiveDict([('author', [Person('Bilderdijk, Willem')]), ('editor', [Person('van Hattum, Marinus')])]))
......................

 80%|████████  | 8/10 [01:22<00:20, 10.28s/it]


DuplicateField: entry with key kijk2019 has a duplicate editor field

In [ ]:
updated_ris[0]

{'type_of_reference': 'Whole book',
 'year': '2014',
 'title': 'Sterke verhalen: vijf eeuwen vertelcultuur',
 'abstract': 'ISBN 978-94-6004-168-6. P. 7-8 Voorwoord, door Garrelt Verhoeven en Jeroen Salman; p. 11-21 Ter introductie: het voortleven van verhalen in woord en beeld, door Jeroen Salman; p. 155-157 Literatuur; p. 158 Verantwoording van het beeld- en bronnenmateriaal. Publicatie ter gelegenheid van de tentoonstelling "Sterke verhalen: vijf eeuwen vertelcultuur" bij de Bijzondere Collecties van de Universiteit van Amsterdam, 24 juni t/m 5 oktober 2014. In het voorjaar van 2014 ontving de Universiteit van Amsterdam een belangrijke schenking van een collectie kinderprenten van verzamelaar Nico Boerma. De waardevolle collectie, met tal van uiterst zeldzame prenten, vormt een belangrijke aanvulling op de verzameling populair drukwerk die al bij de UvA aanwezig was. Prenten uit de collectie Boerma zijn van 24 juni tot en met 5 oktober 2014 te zien bij Bijzondere Collecties UvA op de

In [ ]:
ris_mapping = deepcopy(rispy.TAG_KEY_MAPPING)
ris_mapping['CY'] = 'address'
ris_mapping['T2'] = 'series'

with open(f'llm_test-{ptype}.ris', 'w') as bibliography_file:
    rispy.dump(updated_ris, bibliography_file, mapping=ris_mapping)

In [ ]:
ris_mapping

{'TY': 'type_of_reference',
 'A1': 'first_authors',
 'A2': 'secondary_authors',
 'A3': 'tertiary_authors',
 'A4': 'subsidiary_authors',
 'AB': 'abstract',
 'AD': 'author_address',
 'AN': 'accession_number',
 'AU': 'authors',
 'C1': 'custom1',
 'C2': 'custom2',
 'C3': 'custom3',
 'C4': 'custom4',
 'C5': 'custom5',
 'C6': 'custom6',
 'C7': 'custom7',
 'C8': 'custom8',
 'CA': 'caption',
 'CN': 'call_number',
 'CY': 'place_published',
 'DA': 'date',
 'DB': 'name_of_database',
 'DO': 'doi',
 'DP': 'database_provider',
 'ET': 'edition',
 'EP': 'end_page',
 'ID': 'id',
 'IS': 'number',
 'J2': 'alternate_title1',
 'JA': 'alternate_title2',
 'JF': 'alternate_title3',
 'JO': 'journal_name',
 'KW': 'keywords',
 'L1': 'file_attachments1',
 'L2': 'file_attachments2',
 'L4': 'figure',
 'LA': 'language',
 'LB': 'label',
 'M1': 'note',
 'M3': 'type_of_work',
 'N1': 'notes',
 'N2': 'notes_abstract',
 'NV': 'number_of_volumes',
 'OP': 'original_publication',
 'PB': 'publisher',
 'PY': 'year',
 'RI': 're